In [2]:
import joblib, os
from gensim.models import Word2Vec
import pandas as pd
import datetime

import boto3
from sagemaker import get_execution_role

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
role = get_execution_role()
connection = boto3.client('s3')
contents = connection.list_objects(Bucket='ascsagemaker',
                                  Prefix="JMP_congressional_nmf/dtms")['Contents']

chamber_csv = [f['Key'] for f in contents if f['Key'].endswith('.csv')]
chamber_csv

['JMP_congressional_nmf/dtms/097.csv',
 'JMP_congressional_nmf/dtms/098.csv',
 'JMP_congressional_nmf/dtms/099.csv',
 'JMP_congressional_nmf/dtms/100.csv',
 'JMP_congressional_nmf/dtms/101.csv',
 'JMP_congressional_nmf/dtms/102.csv',
 'JMP_congressional_nmf/dtms/103.csv',
 'JMP_congressional_nmf/dtms/105.csv',
 'JMP_congressional_nmf/dtms/106.csv',
 'JMP_congressional_nmf/dtms/107.csv',
 'JMP_congressional_nmf/dtms/108.csv',
 'JMP_congressional_nmf/dtms/109.csv',
 'JMP_congressional_nmf/dtms/110.csv',
 'JMP_congressional_nmf/dtms/111.csv',
 'JMP_congressional_nmf/dtms/112.csv',
 'JMP_congressional_nmf/dtms/113.csv',
 'JMP_congressional_nmf/dtms/114.csv']

In [4]:
all_speeches = []
for f in chamber_csv:
    if f.endswith('102.csv'):
        all_speeches.extend(pd.read_csv(connection.get_object(Bucket='ascsagemaker',Key=f)['Body'])['speech_processed'])
        print(f)
    
all_speeches = [speech.split() for speech in all_speeches]

JMP_congressional_nmf/dtms/102.csv


In [5]:
print(len(all_speeches))

91261


In [6]:
model = Word2Vec(all_speeches,window=10,sg=1,workers=8)
print(f"\nmodel training time: {str(datetime.timedelta(seconds=model.total_train_time))}")


model training time: 0:03:51.425469


In [7]:
model.save('word2vec-model_102.bin')

In [14]:
print(f"\nmodel training time: {str(datetime.timedelta(seconds=model.total_train_time))}")


model training time: 1:08:45.936644


In [47]:
connection.upload_file('word2vec-model.bin', 'ascsagemaker', 'JMP_congressional_nmf/word2vec-first20/word2vec-first20.bin')
connection.upload_file('word2vec-model.bin.syn1neg.npy', 'ascsagemaker', 'JMP_congressional_nmf/word2vec-first20/word2vec-model.bin.syn1neg.npy')
connection.upload_file('word2vec-model.bin.wv.vectors.npy', 'ascsagemaker', 'JMP_congressional_nmf/word2vec-first20/word2vec-model.bin.wv.vectors.npy')

In [44]:
w2v = gensim.models.Word2Vec.load('word2vec-model.bin')